In [22]:
BYTEORDER = 'big'

In [23]:
# Generate header

width = 2048
width_b = width.to_bytes(4, byteorder=BYTEORDER, signed=False)

height = 2048
height_b = height.to_bytes(4, byteorder=BYTEORDER, signed=False)

channels = 3
channels_b = channels.to_bytes(1, byteorder=BYTEORDER, signed=False)

colorspace = 0
colorspace_b = colorspace.to_bytes(1, byteorder=BYTEORDER, signed=False)

header = b'qoif' + width_b + height_b + channels_b + colorspace_b

In [26]:
with open('out.bin', 'wb') as out_bin:
    out_bin.write(header)

b'qoif\x00\x00\x08\x00\x00\x00\x08\x00\x03\x00'
